# Chapter 2: Large-scale data analysis with spaCy

---

## Data structures : Vocab, Lexemes and StringStore

### Shared vocab and string store
- `Vocab`: store data shared across multiple documents.
- 메모리를 아끼기 위해서 spaCy는 모든 문자열(str)들을 해시값(int)으로 인코딩함.
- 문자열은 `nlp.vocab.strings`라는 객체를 통해 StringStore에 저장됨.
- String store은 양방향 lookup table이다.
    - 물론 shared vocab에 값이 추가된 경우만 reversed됨.

In [1]:
## load model packages
import spacy
nlp = spacy.load('en_core_web_sm')

- coffee는 shared vocab에 등록되지 않음.

In [19]:
'coffee' in set(nlp.vocab.strings)

False

- hashing은 가능하지만 역 연산은 불가능.

In [20]:
coffee_hash = nlp.vocab.strings['coffee']
coffee_string = nlp.vocab.strings[coffee_hash]

KeyError: "[E018] Can't retrieve string for hash '3197928453018144401'. This usually refers to an issue with the `Vocab` or `StringStore`."

- nlp 파이프라인을 통해 coffee가 포함된 document로 한번 처리해주면, shared vocab에 등록이 가능함.

In [27]:
doc = nlp("I love coffee")

In [28]:
print(f"hash value: {nlp.vocab.strings['coffee']}")
print(f"string value: {nlp.vocab.strings[3197928453018144401]}")

hash value: 3197928453018144401
string value: coffee


- shared vocab이 doc이랑도 연결되어 있음.

In [26]:
print(f"hash value: {doc.vocab.strings['coffee']}")

hash value: 3197928453018144401


### Lexemes : entries in the vocabulary
- Lexeme은 **context-independent**인 정보를 attribute로 갖는다.
    - `lexeme.text` : 문자열
    - `lexeme.orth` : hash value
    - `lexeme.is_alpha` : 문자인가 아닌가?
- `nlp.vocab`에 대한 키 값을 통해 lexeme을 얻을 수 있다.

In [32]:
lexeme = nlp.vocab['coffee']

In [33]:
print(lexeme.text, lexeme.orth, lexeme.is_alpha)

coffee 3197928453018144401 True


### Doc, Vocab, StringStore 간의 관계

<img src="https://course.spacy.io/vocab_stringstore.png">

- `Doc`의 값들은 `Token`으로 구성되어 있음.
- `Token`은 vocab에서 해쉬값으로 이루어진 lexeme을 참조한다.
- 단어에 대한 실제 문자열 representation을 얻기 위해서는 string store에서 lookup을 하면된다.

## Data structures : Doc, Span and Token

### Doc Object
- Doc은 nlp object를 처리를 하면 나오는 return 값이다.
- 하지만 Doc을 직접적으로 만들 수도 있다.

In [37]:
# Create an nlp object
from spacy.lang.en import English
nlp = English()

# Import the Doc class
from spacy.tokens import Doc

# The words and spaces to create the doc form
words = ['Hello', 'world', '!']
spaces = [True, False, True]

# create a doc manually
doc = Doc(nlp.vocab, words=words, spaces=spaces)


### Span Object

<img src="https://course.spacy.io/span_indices.png">

- Span은 doc의 slice다.
- Span은 기본적으로 다음과 같은 attributes가 있어야됨.
    - doc_id : span이 referencing하는 document id
    - start : Document에서 span의 시작 idx.
    - end : Document에서 span의 끝 idx.

In [38]:
# Import the Doc and Span clasees
from spacy.tokens import Doc, Span

# The words and spaces to create the doc from
words = ['Hello', 'world', '!']
spaces = [True, False, False]

# Create a doc manually
doc = Doc(nlp.vocab, words=words, spaces=spaces)

# Create a span manually
span = Span(doc, 0, 2)

# Create a span with a label
span_with_label = Span(doc, 0, 2, label="GREETING")

### Best practices
- `Doc`과 `Span`은 토큰과 문서에 대한 관계를 잘 표현하는 객체임.
    - 문자열로는 최대한 나중에 변환하자.
    - 가능하면, token attribute를 사용하자.
- 활용 전에 shared vocab에 passing하자.

---

## Word vectors and semantic similarities

- Doc, Span, Token에 대한 vector representation이 어떻게 계산되는지?
- BERT + spaCy의 best practice?

---

## Combining Models and Rules

### Model-based vs Rule-based

|   | Model-based  | Rule-based  |
|---|---|---|
| Use cases  |  application needs to generalize based on examples  | dictionary with finite number of examples  |
| examples  |  제품명, 사람이름, 주어 목적어 관계 | 국가명, 도시명, 개품종명  |
| spaCy features  | NER, DEP, POS  |  tokenizer, Matcher, PhraseMatcher  |

- 이제 Model-based 시스템이 NLP에서 효과적인 점은 자명하다. BERT!!!
- Rule-based 시스템이 더 효과적인 경우는 분류에 대한 instance가 **static하고, finite할 경우**, 빠르고 간단하게 만들 수 있음.